In [51]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate

import models
import payoffs
import plotting
import selection_functions

In [52]:
%matplotlib inline
plt.style.use("ggplot")

# Basic simulation

In [5]:
integrate.solve_ivp?

In [53]:
# random initial condition
prng = np.random.RandomState(42)
number_of_genotypes = 4
initial_offspring_share, = prng.dirichlet(np.ones(number_of_genotypes), 1)
y0 = initial_offspring_share

# define the selection functions
d1, d3 = 1.5, 1
UGA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d1)
UgA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d3)

# define the payoffs
payoff_kernel = payoffs.prisoners_dilemma_payoffs(prng)

def f(t, y):
    W = models.generalized_sexual_selection(y, UGA, UgA, payoff_kernel)
    y_dot = models.offspring_genotypes_evolution(W, y)
    return y_dot

result = integrate.solve_ivp(f, t_span=(0, 10), y0=y0, rtol=1e-9, atol=1e-12, vectorized=True)

In [54]:
result

  message: 'The solver successfully reached the interval end.'
     nfev: 530
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([  0.00000000e+00,   7.73035243e-03,   8.50338767e-02,
         1.80589388e-01,   2.77094581e-01,   3.74725993e-01,
         4.73415427e-01,   5.73101566e-01,   6.73726179e-01,
         7.75234035e-01,   8.77572804e-01,   9.80692986e-01,
         1.08454925e+00,   1.18912246e+00,   1.29436792e+00,
         1.40024417e+00,   1.50671259e+00,   1.61373734e+00,
         1.72128527e+00,   1.82932584e+00,   1.93783106e+00,
         2.04677537e+00,   2.15613552e+00,   2.26589048e+00,
         2.37602129e+00,   2.48651099e+00,   2.59734446e+00,
         2.70850831e+00,   2.81999079e+00,   2.93178165e+00,
         3.04387207e+00,   3.15625453e+00,   3.26892274e+00,
         3.38187156e+00,   3.49509690e+00,   3.60859565e+00,
         3.72236565e+00,   3.83640559e+00,   3.95071495e+00,
         4.06529401e+00,   4.18014374e+00,   4.

In [58]:
def compute_differences(selection_function, dG, dg, T, R, P, S, M, m, epsilon):

    # random initial condition for first simulation
    prng = np.random.RandomState(42)
    
    # create the selection functions
    if selection_function == "kirkpatrick":
        UGA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, dG)
        UgA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, dg)
    elif selection_function == "seger":
        UGA = lambda x_A: selection_functions.seger_selection(x_A, dG)
        UgA = lambda x_A: selection_functions.seger_selection(x_A, dg)
    elif selection_function == "wright":
        UGA = lambda x_A: selection_functions.wright_selection(x_A, dG)
        UgA = lambda x_A: selection_functions.wright_selection(x_A, dg)
    else:
        valid_funcs = ("kirkpatrick", "seger", "wright")
        msg = "Selection_function must be one of {}, {}, or {}.".format(*valid_funcs)
        raise ValueError(msg)
    # define the payoffs
    payoff_kernel = np.array([[R, S], [T, P]])

    def f(t, y):
        W = models.generalized_sexual_selection(y, UGA, UgA, payoff_kernel, M, m, epsilon)
        y_dot = models.offspring_genotypes_evolution(W, y)
        return y_dot

    # common time points at which solution should be evaluated
    tmin, tmax = (0, 10000)
    ts = np.linspace(tmin, tmax, 100)
    
    # random initial condition
    number_of_genotypes = 4
    initial_offspring_share, = prng.dirichlet(np.ones(number_of_genotypes), 1)
    y0 = initial_offspring_share
    solution1 = integrate.solve_ivp(f, t_span=(tmin, tmax), y0=y0, rtol=1e-9, atol=1e-12, t_eval=ts, vectorized=True)
    
    # generate new random, nearby initial condition for second simulation
    initial_offspring_share, = prng.dirichlet(np.ones(number_of_genotypes), 1)
    y0 = (1e-6 * initial_offspring_share) + (1 - (1e-6 * initial_offspring_share)) * y0
    solution2 = integrate.solve_ivp(f, t_span=(tmin, tmax), y0=y0, rtol=1e-9, atol=1e-12, t_eval=ts, vectorized=True)
    
    diff = np.abs(solution1.y - solution2.y)
    
    return ts, diff
    

def plot_lyapunov_exponents(selection_function, dG, dg, T, R, P, S, M, m, epsilon):

    ts, diff = compute_differences(selection_function, dG, dg, T, R, P, S, M, m, epsilon)
    
    fig, axes = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(12, 8))
    for i in range(4):
        axes[i].plot(ts, diff[i])
        axes[i].set_yscale("log")
        
    return (ts, diff)
    

# Interactive simulation

In [59]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

# sliders used to control the metabolic costs
M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M_G$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m_g$")

# slider used to control which selection function is being used
U_slider = widgets.Dropdown(options=["kirkpatrick", "seger", "wright"], index=0, description=r"$U_{\gamma(j)A}$")

# slider that controls the parameters of the selection function
dG_slider = widgets.FloatSlider(value=1, min=0.0, max=10, step=0.05, description=r"$d_G$")
dg_slider = widgets.FloatSlider(value=1, min=0.0, max=10, step=0.05, description=r"$d_g$")

# slider used to control the mutation rate
e_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-4, description=r"$\epsilon$", readout_format=".4f")

w = widgets.interactive(plot_lyapunov_exponents,
                        selection_function=U_slider, dG=dG_slider, dg=dg_slider, 
                        T=T_slider, R=R_slider, P=P_slider, S=S_slider,
                        M=M_slider, m=m_slider, epsilon=e_slider)
display(w)

A Jupyter Widget

In [19]:
# can get access to the solution!
ts, diff = w.result

In [32]:
from scipy import optimize

In [36]:
def max_exponent(x, selection_function, epsilon):
    dG, dg, T, R, P, S, M, m = x
    ts, diff = compute_differences(selection_function, dG, dg, T, R, P, S, M, m, epsilon)
    exponent = np.max(np.corrcoef(np.log(diff), ts)[-1,:-1])
    return exponent

In [50]:

constraints = [{"type": "ineq", "fun": lambda x: x[2] - x[3]},
               {"type": "ineq", "fun": lambda x: x[3] - x[4]},
               {"type": "ineq", "fun": lambda x: x[4] - x[5]},
               {"type": "ineq", "fun": lambda x: x[6] - x[7]},
               {"type": "ineq", "fun": lambda x: x[5]},
               {"type": "ineq", "fun": lambda x: x[7]},
               {"type": "ineq", "fun": lambda x: min(0.5 * (x[2] + x[5]), x[4]) - x[6]},
               {"type": "ineq", "fun": lambda x: x[0] - x[1]}]
               

x0 = np.array([2, 1, 10, 8, 6, 4, 1, 0.5])
result = optimize.minimize(lambda x, selection_function, epsilon: -max_exponent(x, selection_function, epsilon),
                           x0, args=("kirkpatrick", 1e-3), constraints=constraints, options={"disp": True, "maxiter": 10})

/Users/drpugh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/Users/drpugh/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3087: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/Users/drpugh/anaconda3/lib/python3.6/site-packages/scipy/integrate/_ivp/rk.py:140: RuntimeWarning: invalid value encountered in maximum
  scale = atol + np.maximum(np.abs(y), np.abs(y_new)) * rtol


KeyboardInterrupt: 

In [48]:
result

     fun: -0.6630840114590145
     jac: array([   43153.08966418,   496300.09653103,  1330683.69134033,
        5510700.25715718,  4114374.28457757,  5349557.40669724,
        4907095.94448072,  1580258.93468051])
 message: 'Positive directional derivative for linesearch'
    nfev: 10
     nit: 5
    njev: 1
  status: 8
 success: False
       x: array([  2. ,   1. ,  10. ,   8. ,   6. ,   4. ,   1. ,   0.5])